In [9]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict , Annotated
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import BaseMessage ,SystemMessage,HumanMessage,AIMessage

# Persistance Import
from langgraph.checkpoint.memory import MemorySaver  #kind of memory in langraph which store things in RAM
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
# State Define
from langgraph.graph.message import add_messages
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages ]  # Base Messages : all types of messages is derived from it , it indicate here any type of message can be present either human , AI or system message

# add_messages: is a reducer function like operator.add in previous code.
# Why ?
# Because nature of state is when it get new value its old value (message) is deleted .
# therefore add_messages (reducer function) is  used to maintain conversational history, not forget previous messages or states.

In [ ]:
# LLM define
llm = ChatOpenAI()


In [7]:
def chat_node(state: ChatState):
    # Take User Query From state
    messages = state['messages']

    # Send to llm
    response = llm.invoke(messages)

    # Response store state
    return {'messages' : [response]}

In [8]:
# Graph Define
graph = StateGraph(ChatState)

# nodes add
graph.add_node('chat_node', chat_node)

# Edge add
graph.add_edge(START,'chat_node')
graph.add_edge('chat_node',END)

# Compile
chatbot = graph.compile()


In [ ]:
# Execute Graph
initial_state = {
    'messages' : [HumanMessage(content = "What is Capital of India")]
}

chatbot.invoke(initial_state)

In [ ]:
# Chatbot Logic
while True:
    user_message = input('Type Here')

    print("User:" , user_message)

    if user_message.strip().lower() in ['exit', 'quit' , 'bye']:
        break
    response= chatbot.invoke({'messages': [HumanMessage(content=user_message)]})
    
    print('AI:', response['messages'[-1].content])

Normally
when we reach at end of workflow , our state get erased . every time when we newly invoke (workflow) ,newly state start


But in Persistence
at end of workflow state not get erased and we store it in various places like Databases , memory



In [10]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict , Annotated
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import BaseMessage ,SystemMessage,HumanMessage,AIMessage

# Persistance Import
from langgraph.checkpoint.memory import MemorySaver  #kind of memory in langraph which store things in RAM
from dotenv import load_dotenv

load_dotenv()

True

In [11]:
# State Define
from langgraph.graph.message import add_messages
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages ]  # Base Messages : all types of messages is derived from it , it indicate here any type of message can be present either human , AI or system message

# add_messages: is a reducer function like operator.add in previous code.
# Why ?
# Because nature of state is when it get new value its old value (message) is deleted .
# therefore add_messages (reducer function) is  used to maintain conversational history, not forget previous messages or states.

In [12]:
# LLM define
model = ChatOpenAI()

In [13]:
def chat_node(state: ChatState):
    # Take User Query From state
    messages = state['messages']

    # Send to llm
    response = llm.invoke(messages)

    # Response store state
    return {'messages' : [response]}

In [15]:
# Graph Define

checkpointer = MemorySaver()  # Persistance : memory

graph = StateGraph(ChatState)

# nodes add
graph.add_node('chat_node', chat_node)

# Edge add
graph.add_edge(START,'chat_node')
graph.add_edge('chat_node',END)

# Compile
chatbot = graph.compile(checkpointer=checkpointer)

In [ ]:
# Execute Graph
initial_state = {
    'messages' : [HumanMessage(content = "What is Capital of India")]
}

chatbot.invoke(initial_state)

In [ ]:
# Chatbot Logic
thread_id= '1' # Basically at same time chatbot is used by many peoples . so thread id is chatbot kis people sai kya baat kr rha hai.
while True:
    user_message = input('Type Here')

    print("User:" , user_message)

    if user_message.strip().lower() in ['exit', 'quit' , 'bye']:
        break

    config = {'configurable': {'thread_id':thread_id}}
    response= chatbot.invoke({'messages': [HumanMessage(content=user_message)]},config=config)
    
    print('AI:', response['messages'[-1].content])

In [ ]:
# To view all converstional History
chatbot.get_state(config=config)